In [1]:
%load_ext autoreload
%autoreload 2

##### Please run this code with the kernel reinvent.v3.2

In [2]:
# load dependencies
import sys
import pickle
import os
import shutil
import json
import pandas as pd
import numpy as np
import rdkit.Chem as Chem
from numpy.random import default_rng
import torch
from ast import literal_eval
from torch import nn, optim
from rdkit import Chem
from rdkit.Chem import AllChem
from tdc import Oracle
import subprocess

from utils import fingerprints_from_mol

from scripts.acquisition import select_query

### First, we need to install the custom reinvent scoring package to support the Bradley-Terry model

In [3]:
# print python version
import sys

# Print Python version
print(f"Python version: {sys.version}")

Python version: 3.7.6 | packaged by conda-forge | (default, Jun  1 2020, 18:57:50) 
[GCC 7.5.0]


In [4]:
! pip show reinvent_scoring

Name: reinvent-scoring
Version: 0.0.73
Summary: Scoring functions for Reinvent
Home-page: https://github.com/MolecularAI/reinvent-scoring.git
Author: MolecularAI
Author-email: patronov@gmail.com
License: UNKNOWN
Location: /home/springnuance/reinvent-hitl/reinvent-scoring
Requires: 
Required-by: 


In [5]:
# ! conda install scikit-learn=0.21.3

##### If there already exists reinvent_scoring, we should uninstall it

In [6]:
! pip uninstall -y reinvent_scoring

Found existing installation: reinvent-scoring 0.0.73
Uninstalling reinvent-scoring-0.0.73:
  Successfully uninstalled reinvent-scoring-0.0.73


##### Now we install the custom reinvent scoring package
##### The flag -e means that the package is installed in editable mode, so that changes to the code will be immediately available without reinstalling the package. All package info is stored in the setup.py file.

In [7]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-chemistry"
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-models"

Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Running setup.py develop for reinvent-scoring
Obtaining file:///home/springnuance/reinvent-hitl/reinvent-chemistry
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-chemistry
    Found existing installation: reinvent-chemistry 0.0.51
    Uninstalling reinvent-chemistry-0.0.51:
      Successfully uninstalled reinvent-chemistry-0.0.51
  Running setup.py develop for reinvent-chemistry
Obtaining file:///home/springnuance/reinvent-hitl/reinvent-models
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-models
    Found existing installation: reinvent-models 0.0.15rc1
    Uninstalling reinvent-models-0.0.15rc1:
      Successfully uninstalled reinvent-models-0.0.15rc1
  Running setup.py develop for reinvent-models


In [8]:
# ! conda install -y scikit-learn=0.21.3
! pip list | grep reinvent_scoring

In [28]:
# Testing the function
# feedback_type = "scoring"
# base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/small_drd2_training_data.csv"
# base_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/small_drd2_testing_data.csv"

# # bradley_terry
# feedback_type = "comparing"
# base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/small_drd2_training_data.csv"
# base_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/small_drd2_testing_data.csv"

# rank_listnet
feedback_type = "ranking"
base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/small_drd2_training_data.csv"
base_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/small_drd2_testing_data.csv"

outputs = load_drd2_dataset(feedback_type=feedback_type, data_path=base_training_dataset_path)

In [34]:
from training_Bradley_Terry_model.bradley_terry import BradleyTerryModel
from training_Rank_ListNet_model.rank_listnet import RankListNetModel
from training_Score_Regression_model.score_regression import ScoreRegressionModel
from scripts.acquisition import select_query
from scripts.simulated_expert import ActivityEvaluationModel
from helper import write_REINVENT_config, change_config_json, load_feedback_model, read_scaffold_result

def check_create(path):
    """
    Check if the directory exists, if not, create it.
    """
    if not os.path.exists(path):
        os.makedirs(path)
        
def run_HITL_classify(
        seed, dirname, reinvent_dir, reinvent_env, output_dir, restart,
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        base_testing_dataset_path, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each iteration
        REINVENT_n_steps, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 
        sigma_noise, # noise level for simulated chemist's responses
        threshold # threshold for high scoring molecules
        ):

    np.random.seed(seed)
    rng = default_rng(seed)
    
    ################################################
    # DEFINING REINVENT JOBNAME, JOBID, OUTPUT_DIR #
    ################################################

    jobname = "fine-tune predictive component HITL"
    jobid = f"{dirname}_rounds_{num_rounds}_iters_{num_iters}_queries_{num_queries}_{acquisition}_noise_{sigma_noise}"
    conf_filename = "config.json"

    # create root output dir
    check_create(output_dir)

    # create HITL round folders to store results
    for REINVENT_round in range(1, num_rounds + 1):
        check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}")
        for HITL_iteration in range(1, num_iters + 1):
            check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}")
    
    # Copy the pretrained model to the first HITL_iteration
    shutil.copy2(f"{model_pretrained_path}", f"{output_dir}/REINVENT_round_1/HITL_iteration_1")
    
    # store expert scores
    expert_score = []

    # multi-parameter optimization (MPO) loop
    print(f"Running DRD2 (one objective) with rounds {num_rounds}, iters {num_iters}, queries {num_queries}, seed {seed}. \n Results will be saved at {output_dir}")
    
    # ########################### REINVENT rounds ######################################

    for REINVENT_round in range(1, num_rounds + 1):

        print("=====================================")
        print(f"REINVENT round = {REINVENT_round}")

        REINVENT_round_output_dir = f"{output_dir}/REINVENT_round_{REINVENT_round}"
        
        configuration_JSON_path = write_REINVENT_config(feedback_type, reinvent_dir, jobid, jobname, 
                                                        REINVENT_round_output_dir, conf_filename)


        print(f"Creating config file: {configuration_JSON_path}.")

        configuration = json.load(open(f"{configuration_JSON_path}"))

        current_model_path = f"{REINVENT_round_output_dir}/HITL_iteration_1/{model_pretrained_name}"

        configuration = change_config_json(configuration, REINVENT_n_steps, current_model_path)

        # write the updated configuration file 

        with open(configuration_JSON_path, 'w') as f:
            json.dump(configuration, f, indent=4, sort_keys=True)

        break
    
    #     # generate a pool of unlabelled compounds with REINVENT
    #     print("Run REINVENT")                
    #     command = f"{reinvent_env}/bin/python"
    #     script = f"{reinvent_dir}/input.py"
    #     config_path = configuration_JSON_path
    #     stderr_file = f"{REINVENT_round_output_dir}/run.err"
    #     stdout_file = f"{REINVENT_round_output_dir}/run.out"

    #     # Construct the full command to run
    #     cmd = [command, script, config_path]
    #     # Open the file to which you want to redirect stderr and stdout
    #     with open(stderr_file, 'w') as ferr, open(stdout_file, 'w') as fout:
    #         # Execute the command
    #         result = subprocess.run(cmd, text=True, stdout=fout, stderr=ferr)
    #     # Check the result
    #     print("Exit code:", result.returncode)
        
    #     # Initialize human feedback model
    #     print(f"Loading {feedback_type} feedback model.")

    #     # Loading feedback model
    #     feedback_model_path = f"{REINVENT_round_output_dir}/{model_pretrained_name}"
    #     feedback_model = load_feedback_model(feedback_type, feedback_model_path)
        
    #     #############################################################################
    #     # REINVENT HAS OUTPUT THE RESULT in path f"{REINVENT_round_output_dir}/results" #
    #     #############################################################################
        
    #     output_high_score = read_scaffold_result(f"{REINVENT_round_output_dir}/results/scaffold_memory.csv", threshold)
        
    #     scaffold_output = output_high_score["scaffold"]
    #     smiles_output = output_high_score["smiles"]
    #     human_component_output = output_high_score["human_component"]
    #     raw_human_component_output = output_high_score["raw_human_component"]
    #     total_score_output = output_high_score["total_score"]
        
    #     drd2_oracle_model = ActivityEvaluationModel()
    #     smiles_score_human = [drd2_oracle_model.human_score(smile, sigma_noise) for smile in smiles_output] 
        
    #     # store molecule indexes selected for feedback
    #     selected_feedback = np.empty(0).astype(int)
    #     human_sample_weight = np.empty(0).astype(float)
    #     # store number of accepted queries (y = 1) at each HITL_iteration
    #     n_accept = []

    #     ########################### HITL_iteration in each round #####################
        
    #     for HITL_iteration in range(1, num_iters + 1): # T number of HITL_iterations
    #         print("=====================================")
    #         print(f"HITL_iteration = {HITL_iteration}")
            
    #         # classify 
    #         model = RandomForestClf(fitted_model)
            
    #         if len(smiles) > num_queries:
    #             new_query = select_query(data, num_queries, list(smiles), model, selected_feedback, acquisition, rng) # select n smiles with Active Learning
    #         else:
    #             new_query = select_query(data, len(smiles), list(smiles), model, selected_feedback, acquisition, rng)
            
    #         # Initialize the expert values vector
    #         s_bioactivity = [] # for scores (between 0 and 1)
    #         v_bioactivity = [] # for continuous feedback (regression)
            
    #         # Get expert feedback on selected queries
    #         print(new_query)
    #         for i in new_query:
    #             cur_mol = data.iloc[i]["SMILES"]
    #             print(cur_mol)
    #             value = feedback_model.human_score(cur_mol, sigma_noise)
    #             s_bioactivity.append(value)
            
    #         # Get raw scores and transformed score (if any) from the high scoring molecules in U
    #         raw_scoring_component_names = ["raw_"+name for name in scoring_component_names] 
    #         x_raw = data[raw_scoring_component_names].to_numpy()
    #         x =  data[scoring_component_names].to_numpy()

    #         # get (binary) simulated chemist's responses
            
    #         new_y = np.array([1 if s > 0.5 else 0 for s in s_bioactivity])
    #         accepted = new_y.tolist()
            
    #         expert_score += [accepted]
    #         n_accept += [sum(accepted)]

    #         print(f"Feedback idx at HITL_iteration {REINVENT_round}, {HITL_iteration}: {new_query}")
    #         print(f"Number of accepted molecules at HITL_iteration {REINVENT_round}, {HITL_iteration}: {n_accept[HITL_iteration]}")   
            
    #         # append feedback
    #         if len(new_y) > 0:
    #             selected_feedback = np.hstack((selected_feedback, new_query))

    #         mask = np.ones(N, dtype=bool)
    #         mask[selected_feedback] = False

    #         # use the augmented training data to retrain the model
    #         new_smiles = data.iloc[new_query].SMILES.tolist()
    #         new_mols = [Chem.MolFromSmiles(s) for s in new_smiles]
    #         new_x = fingerprints_from_mol(new_mols, type = "counts")
    #         new_human_sample_weight = np.array([s if s > 0.5 else 1-s for s in s_bioactivity])
    #         sample_weight = np.concatenate([sample_weight, new_human_sample_weight])
    #         print(len(new_x), len(new_y))
    #         x_train = np.concatenate([x_train, new_x])
    #         y_train = np.concatenate([y_train, new_y])
    #         smiles_train = np.concatenate([smiles_train, new_smiles])
    #         print(f"Augmented train set size at HITL_iteration {REINVENT_round}: {x_train.shape[0]} {y_train.shape[0]}")
            
    #         # save augmented training data
    #         D_r = pd.DataFrame(np.concatenate([smiles_train.reshape(-1,1), x_train, y_train.reshape(-1,1)], 1))
    #         D_r.columns = ["SMILES"] + [f"bit{i}" for i in range(x_train.shape[1])] + ["target"]
    #         D_r.to_csv(os.path.join(output_dir, f"augmented_train_set_iter{REINVENT_round}.csv"))

    #         # re-fit and save the model using the augmented train set and save to new directory
    #         model_new_savefile = output_dir + '/{}_HITL_iteration_{}.pkl'.format(predictive_model_name, REINVENT_round)
    #         model._retrain(x_train, y_train, sample_weight = sample_weight, save_to_path = model_new_savefile)
    #         fitted_model = pickle.load(open(model_new_savefile, 'rb'))

    #         # get current configuration
    #         configuration = json.load(open(os.path.join(output_dir, conf_filename)))
    #         conf_filename = "HITL_iteration{}_config.json".format(REINVENT_round)    

    #         # modify model path in configuration
    #         configuration_scoring_function = configuration["parameters"]["scoring_function"]["parameters"]
    #         for i in range(len(configuration_scoring_function)):
    #             if configuration_scoring_function[i]["component_type"] == "predictive_property":
    #                 configuration_scoring_function[i]["specific_parameters"]["model_path"] = model_new_savefile

    #         # Keep agent checkpoint
    #         if REINVENT_round == 1:
    #             configuration["parameters"]["reinforcement_learning"]["agent"] = os.path.join(initial_dir, "results/Agent.ckpt")
    #         else:
    #             configuration["parameters"]["reinforcement_learning"]["agent"] = os.path.join(output_dir, "results/Agent.ckpt")

    #     root_output_dir = os.path.expanduser("{}_seed{}".format(jobid, seed))

    #     # Define new directory for the next round
    #     output_dir = os.path.join(root_output_dir, "HITL_iteration{}_{}".format(REINVENT_round, acquisition))
    #     if not os.path.exists(output_dir):
    #         os.makedirs(output_dir)
    #     print(output_dir)

    #     # modify log and result paths in configuration
    #     configuration["logging"]["logging_path"] = os.path.join(output_dir, "progress.log")
    #     configuration["logging"]["result_folder"] = os.path.join(output_dir, "results")

    #     # write the updated configuration file to the disc
    #     configuration_JSON_path = os.path.join(output_dir, conf_filename)
    #     with open(configuration_JSON_path, 'w') as f:
    #         json.dump(configuration, f, indent=4, sort_keys=True)

    # r = np.arange(len(expert_score))
    # m_score = [np.mean(expert_score[i]) for i in r]
    # print("Mean expert score : ", m_score)

In [57]:
print(os.getcwd())

/home/springnuance/reinvent-hitl/Base-Code-Binh


### Running score regression model

In [ ]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42
dirname = "outputs"
restart = False # If restart is True, we would rerun everything
                # If restart is False, we would continue from the latest found HITL_iteration
                
# change these path variables as required
reinvent_dir = os.path.expanduser("/home/springnuance/reinvent-hitl/Reinvent") # We must use absolute path
reinvent_env = os.path.expanduser("/home/springnuance/miniconda3/envs/ReinventCommunity") # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "scoring" # scoring, comparing, ranking
output_dir = f"output_feedback_{feedback_type}"

# feedback type as scoring:
# Given a molecule, what is the probability that the molecule is active regarding DRD2?  

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/small_drd2_training_data.csv"
base_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/small_drd2_testing_data.csv"
model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/score_regression_model.pth"
model_pretrained_name = "score_regression_model.pth"

num_rounds = 2 # number of rounds, corresponding to R in the paper
num_iters = 3 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 1 # number of REINVENT optimization steps

# Please look at the thompson sampling code and fix it!
acquisition = "thompson" # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 

sigma_noise = 0.0 # noise level for simulated chemist's responses

num_queries = 10 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration
threshold = 0.5 # threshold for high scoring molecules
    
run_HITL_classify(
        seed, dirname, reinvent_dir, reinvent_env, output_dir, restart,
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        base_testing_dataset_path, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of molecules shown at each HITL_iteration to the human for feedback, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
        REINVENT_n_steps, # number of REINVENT optimization steps
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
        sigma_noise, # noise level for simulated chemist's responses
        threshold # threshold for high scoring molecules
)


### Running Bradley Terry model

In [35]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42
dirname = "outputs"
restart = False # If restart is True, we would rerun everything
                # If restart is False, we would continue from the latest found HITL_iteration
                
# change these path variables as required
reinvent_dir = os.path.expanduser("/home/springnuance/reinvent-hitl/Reinvent") # We must use absolute path
reinvent_env = os.path.expanduser("/home/springnuance/miniconda3/envs/ReinventCommunity") # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "comparing" # scoring, comparing, ranking
output_dir = f"output_feedback_{feedback_type}"

# feedback type as comparing:
# Given two molecules, what is the probability that the first molecule is more active than the second molecule regarding DRD2?

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/small_drd2_training_data.csv"
base_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/small_drd2_testing_data.csv"
model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/bradley_terry_model.pth"
model_pretrained_name = "bradley_terry_model.pth"

num_rounds = 2 # number of rounds, corresponding to R in the paper
num_iters = 3 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 1 # number of REINVENT optimization steps

# Please look at the thompson sampling code and fix it!
acquisition = "thompson" # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 

sigma_noise = 0.1 # noise level for simulated chemist's responses

num_queries = 10 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration
threshold = 0.5 # threshold for high scoring molecules
    
run_HITL_classify(
        seed, dirname, reinvent_dir, reinvent_env, output_dir, restart,
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        base_testing_dataset_path, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of molecules shown at each HITL_iteration to the human for feedback, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
        REINVENT_n_steps, # number of REINVENT optimization steps
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
        sigma_noise, # noise level for simulated chemist's responses
        threshold # threshold for high scoring molecules
)


Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring
Running DRD2 (one objective) with rounds 2, iters 3, queries 10, seed 42. 
 Results will be saved at output_feedback_comparing
REINVENT round = 1
Creating config file: output_feedback_comparing/REINVENT_round_1/config.json.


### Running Rank ListNet model

In [ ]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42
dirname = "outputs"
restart = False # If restart is True, we would rerun everything
                # If restart is False, we would continue from the latest found HITL_iteration
                
# change these path variables as required
reinvent_dir = os.path.expanduser("/home/springnuance/reinvent-hitl/Reinvent") # We must use absolute path
reinvent_env = os.path.expanduser("/home/springnuance/miniconda3/envs/ReinventCommunity") # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "ranking" # scoring, comparing, ranking
output_dir = f"output_feedback_{feedback_type}"

# feedback type as ranking:
# Given N molecules, what are the orders of preference of these molecules regarding DRD2?

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/small_drd2_training_data.csv"
base_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/small_drd2_testing_data.csv"
model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/rank_listnet_model.pth"
model_pretrained_name = "rank_listnet_model.pth"

num_rounds = 2 # number of rounds, corresponding to R in the paper
num_iters = 3 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 1 # number of REINVENT optimization steps

# Please look at the thompson sampling code and fix it!
acquisition = "thompson" # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 

sigma_noise = 0.0 # noise level for simulated chemist's responses

num_queries = 10 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration
threshold = 0.5 # threshold for high scoring molecules
    
run_HITL_classify(
        seed, dirname, reinvent_dir, reinvent_env, output_dir, restart,
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        base_testing_dataset_path, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of molecules shown at each HITL_iteration to the human for feedback, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
        REINVENT_n_steps, # number of REINVENT optimization steps
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
        sigma_noise, # noise level for simulated chemist's responses
        threshold # threshold for high scoring molecules
)
